In [1]:
# Import Image from wand.image module
from wand.image import Image

# Read image using Image() function
with Image(filename ="lenna.png") as img:

	# Generate noise image using noise() function
	img.noise("poisson", attenuate = 0.9)
	img.save(filename ="lenna1.png")


In [2]:
##PSNR (Peak Signal-to-Noise Ratio:
#apt install imagemagick
#pip install opencv-python

import math
import numpy as np
import cv2

img_orig = cv2.imread("lenna.png")
img_compress = cv2.imread("lenna1.png") 


def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


print("PSNR is {} dB".format(calculate_psnr(img_orig, img_compress)))

PSNR is 14.463138232110921 dB


In [3]:
#SSIM (Structural Similarity Index)

import math
import numpy as np
import cv2

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

img_orig = cv2.imread("lenna.png")
img_compress = cv2.imread("lenna1.png") 

print("SSIM map mean: {}".format(ssim(img_orig, img_compress)))
print("SSIM value: {}".format(calculate_ssim(img_orig, img_compress)))




SSIM map mean: 0.12679715776948902
SSIM value: 0.12679715776948902


PSNR (Peak Signal-to-Noise Ratio) and SSIM (Structural Similarity Index) are two different metrics used to evaluate the quality of an image or video. They assess quality in slightly different ways, and their primary distinctions are as follows:

Measurement Type:

PSNR: PSNR measures the quality of an image or video by evaluating the amount of noise or distortion present. It calculates the ratio of the peak signal power to the power of the noise. In simple terms, it quantifies how much an image or video has deviated from the original in terms of pixel values.
SSIM: SSIM assesses the structural similarity between the reference (original) and distorted (received or processed) images or videos. It considers not only pixel-wise differences but also structural information such as luminance, contrast, and structure.
Perceptual Consideration:

PSNR: PSNR is a purely mathematical and technical metric. It doesn't consider human perception of image or video quality. A higher PSNR value indicates less distortion, but it may not correlate well with what humans perceive as better quality.
SSIM: SSIM takes into account the human visual system's characteristics. It considers aspects like luminance, contrast, and structural information that align more closely with human perception. Consequently, a higher SSIM value is often associated with higher perceptual quality.
Sensitivity to Compression and Artifacts:

PSNR: PSNR is sensitive to compression artifacts. It can sometimes produce high values for heavily compressed content that may not look good to human viewers.
SSIM: SSIM is more robust in assessing compression and other types of distortions. It often provides a better reflection of perceived quality in these scenarios.
Scale and Range:

PSNR: PSNR is typically measured in decibels (dB), and higher values indicate better quality. A common range is 20-50 dB.
SSIM: SSIM values range from -1 to 1, with 1 indicating perfect similarity. A value of 1 suggests that the reference and distorted images are identical.
In summary, PSNR is a straightforward and widely used metric for measuring image and video quality, but it focuses solely on pixel-wise differences and is less aligned with human perception. SSIM, on the other hand, incorporates structural information and better correlates with how humans perceive quality. When evaluating image or video quality, it's often advisable to consider both metrics, as well as other factors, depending on the specific use case and audience.